In [ ]:
import os
os.chdir("..")  # Ajusta según tu estructura


# Generate_folders_ImageDataProject.py

In [1]:
import os

def create_project_structure():
    """
    Create a folder structure for an image data project.

    The base path is automatically set to the current working directory.
    """
    # Set the base directory to the current working directory
    base_path = os.getcwd()

    # Define the folder structure
    structure = {
        "data": [
            "raw",
            "processed",
            "annotations"
        ],
        "src": [
            "data_preprocessing",
            "models",
            "visualization"
        ],
        "models": [
            "checkpoints",
            "final"
        ],
        "results": [
            "predictions",
            "logs"
        ],
        "docs": [],
        "notebooks": [],
        "tests": [],
        "presentation": []
    }

    # Function to create folders recursively
    def create_folders(base, folders):
        for folder, subfolders in folders.items():
            folder_path = os.path.join(base, folder)
            os.makedirs(folder_path, exist_ok=True)
            for subfolder in subfolders:
                os.makedirs(os.path.join(folder_path, subfolder), exist_ok=True)

    # Create the structure
    create_folders(base_path, structure)

    print(f"Project structure created successfully at: {base_path}")

# Example usage
if __name__ == "__main__":
    create_project_structure()


Project structure created successfully at: c:\Users\User\Projects_Unprotected\LEGO_Bricks_ML_Vision


# Exportar Refurbished PDF

In [4]:
!jupyter nbconvert --to pdf LEGO_Bricks_Identification_Project_Overview.ipynb
# !jupyter nbconvert --to pdf LEGO_Bricks_Identification_Project_Technical_Report.ipynb

[NbConvertApp] Converting notebook LEGO_Bricks_Identification_Project_Overview.ipynb to pdf
[NbConvertApp] Writing 34205 bytes to notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: ['xelatex', 'notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: ['bibtex', 'notebook']
[NbConvertApp] WARNING | b had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 42940 bytes to LEGO_Bricks_Identification_Project_Overview.pdf


# Refurbishing Pipeline

(chatGPT is having bugs remembering converstions)

# "

¡Perfecto! Vamos a estructurar esta entrevista para guiarte a reflexionar sobre cada etapa del proyecto y asegurarnos de que tu storytelling esté alineado con las mejores prácticas. Este enfoque también nos permitirá planear cómo documentar, reproducir y organizar el proyecto de manera profesional.

Entrevista Inicial
Te haré preguntas relevantes por etapas del proyecto. Responde según tu experiencia, y construiré secciones del documento con base en tus respuestas. También sugeriré scripts o ajustes según lo que compartas.

1. Origen del Proyecto
¿Qué te motivó a iniciar este proyecto? ¿Fue un desafío técnico que querías resolver, una curiosidad personal, o algo más?
¿Cómo conectaste tus habilidades previas con esta idea? Por ejemplo, ¿qué experiencias te influyeron a elegir visión por computadora y no otro campo?
2. Creación del Dataset
¿Qué tipo de datos recopilaste y cómo? Comparte detalles sobre el proceso, como herramientas utilizadas, número de imágenes, y estrategias para capturar diferentes ángulos y condiciones.
¿Cómo manejaste la anotación de los datos? ¿Usaste alguna herramienta específica como LabelMe o desarrollaste algo propio? ¿Hubo desafíos en esta etapa?
¿Qué criterios usaste para limpiar y preparar los datos? Por ejemplo, ¿eliminaste imágenes borrosas o hiciste algún balanceo entre clases?
3. Selección del Modelo
¿Por qué elegiste YOLO? ¿Probaste otros modelos antes? Si es así, ¿cuáles y por qué los descartaste?
¿Cómo ajustaste los hiperparámetros durante el entrenamiento? Por ejemplo, tamaño de lotes, número de épocas, etc.
¿Qué limitaciones encontraste en el modelo, y cómo las abordaste?
4. Implementación y Resultados
¿Cómo transformaste los resultados del modelo en algo práctico? Por ejemplo, ¿cómo calculaste las dimensiones de las piezas basándote en la detección de studs?
¿Qué métricas utilizaste para medir el éxito? ¿Precision, recall, F1-score, u otras? ¿Cómo documentaste estos resultados?
¿Qué visualizaciones creaste para entender mejor los resultados?
5. Reflexión y Futuro
¿Qué aprendiste en términos técnicos y personales? ¿Qué habilidades desarrollaste o reforzaste?
Si comenzaras de nuevo, ¿qué harías diferente?
¿Qué pasos futuros te gustaría explorar? Por ejemplo, ¿mejorar el modelo, ampliar el dataset, o implementar nuevas funcionalidades?
Próximos Pasos
Conforme respondas, construiré borradores de las secciones correspondientes del documento. Si surgen necesidades específicas (como scripts para repetir entrenamientos), las desarrollaremos y definiremos cómo integrarlas al proyecto.

Comencemos con tus respuestas a las primeras preguntas de la sección 1 sobre la motivación y el origen del proyecto. 😊

# "

# Implementacion de pipeline central

## pipeline.py

In [ ]:
# # -*- coding: utf-8 -*-

# import os

# def generate_structure_file(output_file="estructura.txt"):
#     """
#     Genera un archivo con la estructura de directorios solo si el directorio actual
#     contiene una carpeta 'src' o está relacionado con ella.

#     Args:
#         output_file (str): Nombre del archivo para guardar la estructura.
#     """
#     # Directorio actual
#     current_dir = os.getcwd()

#     # Verificar si existe la carpeta 'src' en el directorio actual
#     if "src" in os.listdir(current_dir):
#         # Crear la estructura en el directorio actual
#         target_dir = current_dir
#         print(f"Directorio 'src' encontrado en el directorio actual: {target_dir}")
#     else:
#         # Subir al directorio padre y verificar si 'src' es una carpeta hermana
#         parent_dir = os.path.abspath(os.path.join(current_dir, os.pardir))
#         if "src" in os.listdir(parent_dir):
#             # Crear la estructura en el directorio padre
#             target_dir = parent_dir
#             print(f"Directorio 'src' encontrado como carpeta hermana. Generando estructura en: {target_dir}")
#         else:
#             print("No se encontró la carpeta 'src'. No se generará el archivo de estructura.")
#             return

#     # Generar la estructura
#     with open(output_file, "w") as f:
#         for root, dirs, files in os.walk(target_dir):
#             level = root.replace(target_dir, "").count(os.sep)
#             indent = " " * 4 * level
#             f.write(f"{indent}{os.path.basename(root)}/\n")
#             subindent = " " * 4 * (level + 1)
#             for file in files:
#                 f.write(f"{subindent}{file}\n")

#     print(f"Estructura generada exitosamente en: {output_file}")

# # Ejecutar la función
# if __name__ == "__main__":
#     generate_structure_file()


In [4]:
import sys
import os

# Muestra la ruta actual
print(os.getcwd())

# Cambia al directorio raíz del proyecto
os.chdir("..")  # Cambia al directorio raíz

# Confirma el cambio de directorio
print(os.getcwd())

# Importa funciones desde el pipeline
from scripts.pipeline import (
    setup_environment,
    get_kaggle_credentials,
    download_dataset_from_kaggle,
    validate_directories,
    preprocess_images,
    labelme_to_yolo,
    train_yolo_pipeline,
    test_model_on_real_images,
    visualize_results,
)

# Prueba de las funciones importadas
setup_environment()


c:\Users\User\Projects_Unprotected\LEGO_Bricks_ML_Vision
c:\Users\User\Projects_Unprotected
Using device: cpu


# Overview.pdf

## visualize_presentation.py

In [ ]:
# import os
# import random
# import matplotlib.pyplot as plt
# from PIL import Image
# from pipeline import test_model_on_real_images

# # Crear grids de muestras del dataset
# def create_dataset_grid(input_folder, output_folder, grid_size=(3, 3)):
#     """
#     Genera un grid de imágenes de muestra del dataset y las guarda en la carpeta especificada.

#     Parameters:
#     - input_folder (str): Ruta de la carpeta con las imágenes de entrada.
#     - output_folder (str): Ruta donde se guardará el grid generado.
#     - grid_size (tuple): Dimensiones del grid (filas, columnas).
#     """
#     os.makedirs(output_folder, exist_ok=True)
#     image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png'))]
#     random.shuffle(image_files)
    
#     selected_files = image_files[:grid_size[0] * grid_size[1]]
#     fig, axes = plt.subplots(*grid_size, figsize=(grid_size[1] * 4, grid_size[0] * 4))

#     for idx, img_file in enumerate(selected_files):
#         img_path = os.path.join(input_folder, img_file)
#         img = Image.open(img_path)
#         row, col = divmod(idx, grid_size[1])
#         axes[row, col].imshow(img)
#         axes[row, col].axis('off')
#         axes[row, col].set_title(f"{img_file}", fontsize=10)

#     for ax in axes.flat[len(selected_files):]:
#         ax.axis('off')

#     plt.tight_layout()
#     grid_path = os.path.join(output_folder, "dataset_grid.png")
#     plt.savefig(grid_path, bbox_inches='tight')
#     plt.close()
#     print(f"Grid de dataset guardado en {grid_path}.")

# # Generar visualizaciones anotadas
# def annotate_model_results(model_path, input_folder, output_folder, conf_threshold=0.5):
#     """
#     Genera imágenes con anotaciones del modelo y las guarda en la carpeta especificada.

#     Parameters:
#     - model_path (str): Ruta al modelo YOLO entrenado.
#     - input_folder (str): Carpeta con imágenes de entrada.
#     - output_folder (str): Carpeta para guardar las imágenes anotadas.
#     - conf_threshold (float): Umbral de confianza para las detecciones.
#     """
#     os.makedirs(output_folder, exist_ok=True)
#     test_model_on_real_images(model_path, input_folder, output_folder)
#     print(f"Resultados anotados guardados en {output_folder}.")

# # Crear comparaciones antes/después
# def generate_comparison_grid(model_path, input_folder, output_folder, num_samples=5):
#     """
#     Crea un grid comparativo de imágenes antes y después de las predicciones del modelo.

#     Parameters:
#     - model_path (str): Ruta al modelo YOLO entrenado.
#     - input_folder (str): Carpeta con imágenes de entrada.
#     - output_folder (str): Carpeta para guardar los grids generados.
#     - num_samples (int): Número de imágenes a comparar.
#     """
#     os.makedirs(output_folder, exist_ok=True)
#     image_files = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.png'))]
#     random.shuffle(image_files)
#     selected_files = image_files[:num_samples]

#     for img_file in selected_files:
#         img_path = os.path.join(input_folder, img_file)
#         result_path = os.path.join(output_folder, img_file)

#         test_model_on_real_images(model_path, img_path, output_folder)
#         annotated_img = Image.open(result_path)
#         original_img = Image.open(img_path)

#         fig, axes = plt.subplots(1, 2, figsize=(10, 5))
#         axes[0].imshow(original_img)
#         axes[0].axis('off')
#         axes[0].set_title("Original")

#         axes[1].imshow(annotated_img)
#         axes[1].axis('off')
#         axes[1].set_title("Anotado")

#         plt.tight_layout()
#         comparison_path = os.path.join(output_folder, f"comparison_{img_file}")
#         plt.savefig(comparison_path, bbox_inches='tight')
#         plt.close()
#         print(f"Grid de comparación guardado en {comparison_path}.")

# # Organización automática de subcarpetas
# def organize_presentation_folders(base_folder):
#     """
#     Crea y organiza las subcarpetas necesarias para las visualizaciones.

#     Parameters:
#     - base_folder (str): Carpeta raíz para las subcarpetas.
#     """
#     subfolders = ["dataset_samples", "model_results", "before_after"]
#     for subfolder in subfolders:
#         os.makedirs(os.path.join(base_folder, subfolder), exist_ok=True)
#     print(f"Subcarpetas creadas en {base_folder}: {', '.join(subfolders)}")

# # Ejecución del pipeline de visualización
# def main():
#     base_folder = "presentation"
#     organize_presentation_folders(base_folder)

#     dataset_folder = "datasets/processed_images"
#     model_path = "YOLO_Lego_Detection/best.pt"

#     create_dataset_grid(dataset_folder, os.path.join(base_folder, "dataset_samples"))
#     annotate_model_results(model_path, dataset_folder, os.path.join(base_folder, "model_results"))
#     generate_comparison_grid(model_path, dataset_folder, os.path.join(base_folder, "before_after"))

# if __name__ == "__main__":
#     main()


In [ ]:
import sys
import os

# Muestra la ruta actual
print(os.getcwd())

# Cambia al directorio raíz del proyecto
os.chdir("..")  # Cambia al directorio raíz

# Confirma el cambio de directorio
print(os.getcwd())

sys.path.append(os.getcwd())

# Importar funciones clave
from visualize_presentation import (
    create_dataset_grid,
    annotate_model_results,
    generate_comparison_grid,
    organize_presentation_folders
)

# Definir rutas de entrada y salida
BASE_FOLDER = "presentation"
DATASET_FOLDER = "datasets/processed_images"
MODEL_PATH = "YOLO_Lego_Detection/best.pt"

# Crear y organizar las carpetas de presentación
organize_presentation_folders(BASE_FOLDER)


c:\Users\User\Projects_Unprotected
c:\Users\User


ModuleNotFoundError: No module named 'visualize_presentation'

In [1]:
import sys
import os
from pprint import pprint

# Muestra la ruta actual
print(os.getcwd())

# Cambia al directorio raíz del proyecto
os.chdir("..")  # Cambia al directorio raíz

# Confirma el cambio de directorio
print(os.getcwd())

sys.path.append(os.getcwd()) 

pprint(sys.path)




c:\Users\User\Projects_Unprotected\LEGO_Bricks_ML_Vision\notebooks
c:\Users\User\Projects_Unprotected\LEGO_Bricks_ML_Vision
['c:\\Users\\User\\Projects_Unprotected\\LEGO_Bricks_ML_Vision\\notebooks',
 'c:\\Users\\User\\.conda\\envs\\MiguelEnvHaB\\python312.zip',
 'c:\\Users\\User\\.conda\\envs\\MiguelEnvHaB\\DLLs',
 'c:\\Users\\User\\.conda\\envs\\MiguelEnvHaB\\Lib',
 'c:\\Users\\User\\.conda\\envs\\MiguelEnvHaB',
 '',
 'C:\\Users\\User\\AppData\\Roaming\\Python\\Python312\\site-packages',
 'c:\\Users\\User\\.conda\\envs\\MiguelEnvHaB\\Lib\\site-packages',
 'c:\\Users\\User\\.conda\\envs\\MiguelEnvHaB\\Lib\\site-packages\\win32',
 'c:\\Users\\User\\.conda\\envs\\MiguelEnvHaB\\Lib\\site-packages\\win32\\lib',
 'c:\\Users\\User\\.conda\\envs\\MiguelEnvHaB\\Lib\\site-packages\\Pythonwin',
 'c:\\Users\\User\\.conda\\envs\\MiguelEnvHaB\\Lib\\site-packages\\setuptools\\_vendor',
 'c:\\Users\\User\\Projects_Unprotected\\LEGO_Bricks_ML_Vision']


In [ ]:
# Ejecutar todo el pipeline de visualización
def run_visualization_pipeline():
    organize_presentation_folders(BASE_FOLDER)
    create_dataset_grid(DATASET_FOLDER, f"{BASE_FOLDER}/dataset_samples", grid_size=(3, 3))
    annotate_model_results(MODEL_PATH, DATASET_FOLDER, f"{BASE_FOLDER}/model_results")
    generate_comparison_grid(MODEL_PATH, DATASET_FOLDER, f"{BASE_FOLDER}/before_after", num_samples=5)

run_visualization_pipeline()
